## This notebook demonstrates how to perform bulk inference of [Phi-3-mini-128k-instruct](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct) on the [Tracto.ai](https://tracto.ai/) platform.

Tracto is perfect for offline batch inference:

* Easy scaling - just change the `job_count` parameter.
* Fault tolerance out of the box - if a job crashes, it gets restarted automatically. No need to handle GPU failures or infrastructure issues - Tracto takes care of it.
* Full Tracto integration - save results directly to the distributed file system and process them further on the platform.

In [1]:
# configure environment to run this notebooks
import uuid
import yt.wrapper as yt

username = yt.get_user_name()
if yt.exists(f"//sys/users/{username}/@user_info/home_path"):
    # prepare working directory on distributed file system
    user_info = yt.get(f"//sys/users/{yt.get_user_name()}/@user_info")
    homedir = user_info["home_path"]
    # find avaliable vm presets
    cpu_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("cpu")] or ["default"]
    h100_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h100")]
    h100_8_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h100-8")]
    h200_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h200")]
    h200_8_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h200-8")]
    workdir = f"{homedir}/tmp/demo_workdir/{uuid.uuid4().hex}"
else:
    cpu_pool_trees = ["default"]
    h100_pool_trees = ["gpu_h100"]
    h100_8_pool_trees = ["gpu_h100"]
    h200_pool_trees = ["gpu_h200"]
    h200_8_pool_trees = ["gpu_h200"]
    workdir = f"//tmp/examples/{uuid.uuid4().hex}"

yt.create("map_node", workdir, recursive=True, ignore_existing=True)
print("Current working directory:", workdir)

Current working directory: //home/impossible_teal_dog/tmp/demo_workdir/9ec2ec408b9144aab457463d7997e24b


Prepare data for inference as a table.

In [3]:
from datasets import load_dataset

dataset = load_dataset("Rapidata/Other-Animals-10")

table_path = f"{workdir}/questions"
yt.create("table", table_path, force=True)

questions = [
    {"question": f"Can a {animal} fly? If it can't - how can we help it take off?"}
    for animal in set(dataset["train"].features["label"].int2str(dataset["train"]["label"]))
]

yt.write_table(table_path, questions)

Run bulk inference of `Phi-3-mini`.

In [5]:
from typing import Iterable
import logging
import sys
import random

yt.config["pickling"]["safe_stream_mode"] = False


result_path = f"{workdir}/result"


@yt.aggregator
def bulk_inference(records: Iterable[dict[str, str]]) -> dict[str, str]:
    from vllm import LLM, SamplingParams

    llm = LLM(model="microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)
    sampling_params = SamplingParams(
        temperature=0.6,
        top_p=0.9,
        max_tokens=32000,
    )

    conversations = [
        [
            {
                "role": "user",
                "content": record["question"],
            },
        ]
        for record in records
    ]
    outputs = llm.chat(
        messages=conversations,
        sampling_params=sampling_params,
    )
    for prompt, output in zip(conversations, outputs):
        yield {
            "prompt": prompt,
            "text": output.outputs[0].text,
        }


yt.run_map(
    bulk_inference,
    table_path,
    result_path,
    job_count=2,
    spec={
        "pool_trees": h100_pool_trees,
        "mapper": {
            "gpu_limit": 1,
            "memory_limit": 16106127360,
            "cpu_limit": 10,
            "environment": {
                # improtant for vllm v1
                "VLLM_WORKER_MULTIPROC_METHOD": "spawn",
            },
        },
    },
)

2025-07-03 20:12:46,067	INFO	Operation started: https://playground.tracto.ai/playground/operations/df7ff07b-d9bd119a-24dd03e8-33f3536b/details


2025-07-03 20:12:46,137	INFO	( 0 min) operation df7ff07b-d9bd119a-24dd03e8-33f3536b initializing


2025-07-03 20:12:47,360	INFO	( 0 min) Unrecognized spec: {'enable_partitioned_data_balancing': false, 'mapper': {'title': 'bulk_inference'}}


2025-07-03 20:12:48,055	INFO	( 0 min) operation df7ff07b-d9bd119a-24dd03e8-33f3536b: running=0     completed=0     pending=2     failed=0     aborted=0     lost=0     total=2     blocked=0    


2025-07-03 20:12:51,211	INFO	( 0 min) operation df7ff07b-d9bd119a-24dd03e8-33f3536b: running=1     completed=0     pending=1     failed=0     aborted=0     lost=0     total=2     blocked=0    


2025-07-03 20:14:23,070	INFO	( 1 min) operation df7ff07b-d9bd119a-24dd03e8-33f3536b: running=1     completed=1     pending=0     failed=0     aborted=0     lost=0     total=2     blocked=0    


2025-07-03 20:15:55,285	INFO	( 3 min) operation df7ff07b-d9bd119a-24dd03e8-33f3536b completed


In [6]:
for record in yt.read_table(result_path):
    print(f"Q: {record['prompt']}")
    print(f"A: {record['text']}")

Q: [{'role': 'user', 'content': "Can a wildboar fly? If it can't - how can we help it take off?"}]
A:  No, a wild boar cannot fly. Wild boars are terrestrial mammals that belong to the family Suidae. They are heavy-set animals with short legs and a stout body, adapted for a life on land. Their physical characteristics, such as their large, heavy bodies and lack of wings, make flight impossible.

However, if you're concerned about a wild boar's safety or well-being, there are several ways you can help ensure their survival:

1. Habitat conservation: Protecting and preserving their natural habitats, such as forests, grasslands, and wetlands, is crucial for their survival. This can be done through supporting conservation organizations, participating in local habitat restoration projects, or advocating for responsible land-use practices.

2. Reduce human-wildlife conflicts: Wild boars can cause damage to crops, gardens, and property when they come into contact with humans. To minimize thes